## Text Generation with LSTM using Lord of the Rings: The Two Towers corpus 

https://medium.com/@matthewbejtlich/text-generation-with-lstm-lord-of-the-rings-the-two-towers-99e977b8a5e8

## Model 1

This is the first model for my text generation with LSTM. Please see Text_Generation_Model_2.ipynb for Model 2.

In [14]:
import keras
keras.__version__

'2.1.5'

### Load Lord of Rings: The Two Towers corpus

In [15]:
import numpy as np

text = open('data/Lordoftherings2.txt').read().lower()
print('Corpus length:', len(text))
#new = text.split(" ")
#print(new)

Corpus length: 834860


In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 278267
Unique characters: 58
Vectorization...


### Building the network

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_1 (Dense)              (None, 58)                7482      
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Training language model and sampling 

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
278267/278267 [==============================] - 180s 647us/step - loss: 1.8254
--- Generating with seed: "great ridge. there seems to be a smoke or haze over there, l"
------ temperature: 0.2
great ridge. there seems to be a smoke or haze over there, lith and the shade of the streak of the shade the moures of the right of the shade of the great were and the strange of the hard be the came the mouth and the streak the shade the shade the streak of the shade the streams and the shade the shade and the singing the shade the shade the strange of the great the dark of the streak to the streak the strange of the shade the mouth and there was the stre
------ temperature: 0.5
ak the strange of the shade the mouth and there was the stree, the manches 
more and there the was 
them, there he are go the treeping not 
the streak the ores 
was stort he 
were him the dourntards and had the north them and the shade and at live a mind entle of the great 
plate and a fing him bears of

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


stream the way of the sun and 
strength the stream was a shadows of the way of the river of the thing of the stream to the stream to the stream of the some of the silence of the some and stones the end and the mountains of the some of the s
------ temperature: 0.5
me and stones the end and the mountains of the some of the spear of the rivered and walk to them. if it was 
the eastern had pask to the rock some coming and drawing away where let it and was far away forward the filldering of the grass and 
more of 
one of a live the hir worse was closes and a wizard that he may me the hobbit know him and the way of the hobbit mouth to the hobbit promised. 'he was a shadows with not shall of silence of 
this werss and the
------ temperature: 1.0
s a shadows with not shall of silence of 
this werss and there hobbit to see in 
the riw horse? i have usonot that his greys in his 
fearonsed, unindard alroad mastered without has pass away the some dark laughed on passed to flaels round 

preat mil

'i mark them,' said aragorn. 'but they were and the stones of the stones of the 
stones of the stream they were and the 
stones of the stones of the stream of the stream of the 
long 
stream and the stream of the 
trees of the mountains of the 
stream of the stones of the great black stones and the trees, and the trees of the end of the stream of the 
stones of the stones of the 
way of the trees, and the way of the 

------ temperature: 0.5
of the stones of the 
way of the trees, and the way of the 
shadows of the briesten of the feet and he took to the enemy. 

'i said aragorn 
he took 
them. the north was some thought he said them, and the storm in the mouth of the 
long of the woods of the guards of the 
watchers and pitping for they were been and for a long stood their horses, and they were spears and 
their eyes to 
his hiss, and sam had a 
stair,' said gimli. 'i do not died at like a la
------ temperature: 1.0
d sam had a 
stair,' said gimli. 'i do not died at like a last alive 

thought is night. after the deay d nomteres ))ness: his tree! is they w)ward what anothers's uncelping far 
off. i truatled him precious on the fords, at )glit.' 

h
------ temperature: 1.2
r 
off. i truatled him precious on the fords, at )glit.' 

her ore as the edge. out neverf, . not?', wonf a smell flink.  picked changed. the damter saruman was softly bodian)o, 'fover), an)?' drawny or sween of a)d from c)))! fimy lall, i )ds!' said gollum. 'post in rump nazey forsolk. bunb uce.' 

in the feet )in2)-and wless, theml tcourse ei).). 

a chiefing!' said not folpotsing. he was brools with after the). there goll on time. but eithere eaot. we e
epoch 16
Epoch 1/1
278267/278267 [==============================] - 174s 626us/step - loss: 2.1000
--- Generating with seed: "s the long valley came, 
falling steeply in an ever-widening"
------ temperature: 0.2
s the long valley came, 
falling steeply in an ever-widening of the). the arms) the m), an). 'well, the mountain). 

'wel). 

'well, well

compan) of ores and sle)tbess, and the 
find )gw) unlerged ))). 'we fountry. 'come th)izad itum. 'to iseliming 
us as it safe yo))ead awr)))eat lost tore of end with anoth: our by the 
sit it. 'do, it seemed step upon the dark 
)cts )crt. 

'stfurus, you 
satseen i. gollum 
much foundror ale men. 'we are us and), with himing it had ansked))ed 
on the rang. 




allows it drung. a long 
stranged path are formmalishoness-fare)ned in th
------ temperature: 1.2
ng. a long 
stranged path are formmalishoness-fare)ned in the cores) by  cauld thppe and low edresh)), i cuset off. he )l?’ 

and one)ce 


), one scorors, may clay. fom t)nint was a whistzed came whf wa)n accors of 
. the red))wpy has red upof without thst from the )creat; corcked hiding and slopot v)nke. ))l!' are no) old ord 
green t)ved; even they cu)ve toy frofting, because of a thinurilils the crike s)ounith, no) the)er 
froness tim intoses with ten 
epoch 20
Epoch 1/1
278267/278267 [==============================] - 174s 625u

'i do not do not to the strange the strange the strange the stream and the strong of the stream and spears and strong and the stream of the mountains and the strange the strange the strange and a strange and a stream and still the strange and the stream the strange the thing of the stream and spoke and the strange the stream and spoke that see the strange and spoke the stream the stone o
------ temperature: 0.5
 spoke that see the strange and spoke the stream the stone of the stream was days of the markming stream. 




'he was been and isengard it call at the river under 
the ores, and the ores straight and passed the fool. 

'there were shall to watched the stranges was give a long 
the 
look and distane to many some bean hand. and shadowfell were hand the speed 
over the stones of the east. the fight was the walls of the sign for 
the sign and clear the spring
------ temperature: 1.0
was the walls of the sign for 
the sign and clear the springs. ass! then they fell diid over, foes g

far cle that though were more plain, outsing iunt bmardted's thought back onder oroo's young drow an brugge whit 
him, abouts the gradious, 
vistheroul in feamief away it was them. 

'ye dugon that journery arr was three it. 

'whate the enpice starly. alsug where heh enow its 
tven neck. 

bringsor rowys are not foner and anyahon, either give alay were light, preciove. sick alay 
noesing appeared the darknon
epoch 35
Epoch 1/1
278267/278267 [==============================] - 174s 625us/step - loss: 1.5667
--- Generating with seed: " looked up. a dark water was before him, and he was crawling"
------ temperature: 0.2
 looked up. a dark water was before him, and he was crawling the strange and the strange had could saruman the storpled and the strange and stort the 
strange had been 
sade and the strange the strange of the strange the stort they said the strange and the strange and the strange and the sundd and the strange they said and the hobbits and the 
strange of the reant strange 

t osd )r— t  tlorr a bhorn wqtt —mare teale) aall toou  ae tas  are bictinl( saq heqtoo toene hiu 1  he  as to )n li 
q haed wae s ty t t th—es t®r t) thee  too — tourn fhoe  tir hy t"   
and  hau lqou t  toi a woosi me   tras toe he tha
------ temperature: 1.0
r hy t"   
and  hau lqou t  toi a woosi me   tras toe he tha  t 
’red t  —el) ta tyi) ioil —aallne  entar "ed cyue  si— bey — aro ma® aittoe t are tri s an aqo 
"dodoroa^me 

as— ma4
e  home roulenes tu rimar 

asere w a rama a s  quatuoetoa  omoqgr t storqert   wit" a sale 
 s)ta the ante innenaand a roi doibe  toae qer saanen soea  aat)  i mea  s woer re i2d ioso wbe ure ha i®ro ’s
'
ootrr dar e loa taae yor a ine"s te a to  are 

 han
as an t— 
lei sre 
------ temperature: 1.2
 loa taae yor a ine"s te a to  are 

 han
as an t— 
lei sre ranre qundeom — jdo'men) raa  anos sasure  olek  ho 0no  and tquoet uht the a i s
 doals  eri t rin)h hasen t s—'si  ho )ur elotre rd1 hinde dre (hine  in hoaq) ot, s hd ' thael s 
 a  sd w lyo

fancy. r  re  ar     ior e    ae  oie e oa  e  a re  a  o     
 s i g or grrer i  r ar a ee  e aa o  aae eee oe ae  ar   aere e  a e re al  i  a   a  a   aic  ra  n ae re eoeerle  a ai   a   a  e  o  eeeee a  a te e  a  ooi arr  a  oeer  orre  a  i  er e  ae  
  eai    an   orire e ae i  ar  rer    re  a  aeeer ai i  ao e rre  are    a  aat  or   tee a    ore  e  a  o   rere  i   a  e    t a f e are ee 
------ temperature: 0.5
   tee a    ore  e  a  o   rere  i   a  e    t a f e are ee rrr a or oe t     ore iee  a  ia re oaeee  are"oa   a  roo i awere  e  'aiee   r   r oaro oe  eaore  la a i  ere   a arer   iraiirea oo e a areii o a  o  o  i   aroe   oe re  a oa  a oee  ieo vrar oare iee ore  t ore  r aa oa     ro  ei ear    iei  a   teoore  ar ar  ea 're   ai ael   io r a i  r roaer    a  aa  i a  a   o  at o ba a are a   arr ae    o erha i arre    e ai i a    re  ie o
a  araa 
------ temperature: 1.0
a   arr ae    o erha i arre    e ai i a    re  ie o
a  araa  e  ie  i ie  rara j  e 

### Model 2

In [16]:
# Length of extracted character sequences
maxlen = 100

# We sample a new sequence every `step` characters
step = 5

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 166952
Unique characters: 58
Vectorization...


In [17]:
from keras import layers
from keras import regularizers

model2 = keras.models.Sequential()
model2.add(layers.LSTM(128, input_shape=(maxlen, len(chars)))) 
model2.add(layers.Dense(len(chars), activation='softmax'))

In [18]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_3 (Dense)              (None, 58)                7482      
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model2.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [20]:
import random
import sys

for epoch in range(1, 25):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model2.fit(x, y,
              batch_size=128,
              epochs=1)
    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1


InternalError: GPU sync failed